In [1]:
import matplotlib.pyplot as plt

import pyomo.environ as pyo
from pyomo.environ import value

import numpy as np

# Minimax Estimation

## Hacks

We set $\mathbb S = [1, \ldots 1000]$ and $J=10$, $\mathbb S_1 = [1:100]$, $\mathbb S_2 = [101:200]$ and so on.

In [ ]:
def sum_power(x, n, k=2):
    return sum(x[i] ** k for i in range(n))

def get_p_in_bucket(m, j):
    return m.__getattribute__("p_in_bucket{0}".format(j))

def get_pis(m):
    return [sum([get_p_in_bucket(m, j)[i] for i in range(K)]) for j in range(J)]
    #return [pyo.summation(get_p_in_bucket(m, j)) for j in range(J)]

def sum_ps(m, k):
    tmp = 0
    for j in range(J):
        aux = get_p_in_bucket(m, j)
        tmp += sum(aux[i] ** k for i in range(K))
    return tmp

def weighted_sum_ps(m, pis, k):
    tmp = 0
    for j in range(J):
        aux = get_p_in_bucket(m, j)
        tmp += pis[j] * sum(aux[i] ** k for i in range(K))
    return tmp
        

def risk_fn(m):
    beta = m.beta[0]
    out = 0
    pis = get_pis(m)
    out = beta * beta * (
        n * sum_power(pis, J, 2) + n * (n-1) * sum_power(pis, J, 3))
    out -= 2 * beta * (
        n * sum_ps(m, 2) + n * (n-1) * weighted_sum_ps(m, pis, 2))
    out += n * sum_ps(m, 2) + n * (n-1) * sum_ps(m, 3)
    return out


def risk_fn_debug(p, beta):
    def wsum_p(pis, p):
        out = 0
        for j in range(J):
            out += pis[j] * np.sum(p[j * K : (j+1) * K] ** 2)
        return out
    
    pis = np.array([
        np.sum(p[j * K : (j+1) * K]) for j in range(J)])
    print("pis: ", pis)
    
    out = beta * beta * (
        n * sum_power(pis, J, 2) + n * (n-1) * sum_power(pis, J, 3))
    out -= 2 * beta * (
        n * np.sum(p ** 2) + n * (n-1) * wsum_p(pis, p))
    out += n * np.sum(p ** 2) + n * (n-1) * np.sum(p ** 3)
    return out

In [ ]:
def inner_opt(m):
    return m.z[0]


def stupid_initialize(args):
    out = np.zeros(K) + 1e-4
    out[0] = 1 / J
    return out


def fit(K, J, n):
    
    model = pyo.ConcreteModel()
    init_val = np.random.uniform(size=K)
    init_val = init_val / np.sum(init_val)
    init_val /= J

    for j in range(J):
        model.add_component(
            "p_in_bucket{0}".format(j), 
            pyo.Var(np.arange(K), domain=pyo.NonNegativeReals,
                initialize=init_val.copy()))

    model.beta = pyo.Var(np.arange(1), domain=pyo.Reals, initialize=np.random.uniform())
    model.z = pyo.Var(np.arange(1), domain=pyo.Reals, initialize=np.ones(1) * 1e6)

    model.obj = pyo.Objective(rule=inner_opt)
    model.costr = pyo.ConstraintList()

    tmp = 0
    for j in range(J):
        for i in range(K):
            tmp += model.__getattribute__("p_in_bucket{0}".format(j))[i]

    # sum to 1 constraint
    model.costr.add(tmp == 1.0)

    # minimax trick
    model.costr.add(model.z[0] >= risk_fn(model))
    
    solver = pyo.SolverFactory("multistart")
    out = solver.solve(model, suppress_unbounded_warning=True)
    
    if out["Solver"][0]["Status"] != "ok":
        return 0, np.zeros(K * J)
    
    beta = model.beta.extract_values()[0]
    ps = np.zeros(K * J)
    for j in range(J):
        for i in range(K):
            ps[K * j + i] = pyo.value(model.__getattribute__("p_in_bucket{0}".format(j))[i])
    return beta, ps

In [ ]:
n = 1000.0
Ks = [10, 25, 50, 100]
Js = [10, 25, 50]

dist_from_unif = []
betas = []

for J in Js:
    curr_dists = []
    curr_betas = []
    for K in Ks:
        ntry = 0
        sumP = 0
        
        while ntry < 100 and np.abs(sumP - 1) > 0.1:
            try:
                beta, ps = fit(K, J, n)
                sumP = np.sum(ps)
                print("J: {0}, K: {1}, sumP: {2}, ntry: {3}".format(J, K, np.sum(sumP), ntry))
                ntry +=1
            except Exception as e:
                ntry +=1
        
        curr_betas.append(beta)
        
        p_unif = np.ones(K * J)
        p_unif /= np.sum(p_unif)
        curr_dists.append(np.sum(np.abs(ps - p_unif)))
    
    betas.append(curr_betas)
    dist_from_unif.append(curr_dists)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=len(Js), figsize=(12, 3))

krange = np.linspace(Ks[0], Ks[-1], 100)

for i, ax in enumerate(axes):
    ax.plot(krange, 1.0 / krange, color="red")
    ax.set_title("J: {0}".format(Js[i]), fontsize=18)
    ax.scatter(Ks, betas[i])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=len(Js), figsize=(12, 3))

for i, ax in enumerate(axes):
    ax.set_title("J: {0}".format(Js[i]), fontsize=18)
    ax.scatter(Ks, dist_from_unif[i])
    Ks = np.array(Ks)
    ax.plot(Ks, np.zeros(len(Ks)), color="red")
    ax.set_ylim(-1e-6, 1e-6)

# Worst Distribution for Fixed Beta

In [ ]:
def inner_opt(m):
    return - risk_fn(m)


model = pyo.ConcreteModel()

for j in range(J):
    init_val = np.zeros(K)
    
    if j == 0:
        init_val[0] = 1.0
        
    model.add_component(
        "p_in_bucket{0}".format(j), 
        pyo.Var(np.arange(K), domain=pyo.NonNegativeReals,
            initialize=init_val.copy()))

model.beta = pyo.Var(np.arange(1), domain=pyo.Reals, initialize=0.5)
model.z = pyo.Var(np.arange(1), domain=pyo.Reals, initialize=np.ones(1) * 1e3)

model.obj = pyo.Objective(rule=inner_opt)
model.costr = pyo.ConstraintList()

tmp = 0
for j in range(J):
    for i in range(K):
        tmp += model.__getattribute__("p_in_bucket{0}".format(j))[i]

# sum to 1 constraint
model.costr.add(tmp == 1.0)

In [ ]:
value(inner_opt(model))

In [ ]:
model.beta.fix(0.1)
solver = pyo.SolverFactory("ipopt")
out = solver.solve(model)
print(out)

In [ ]:
value(inner_opt(model))

# Upper Bound

In [ ]:
beta = 0.5

def upper_bound(model):
    return beta ** 2 * n * model.A[0] + n * (n-1) * (beta**2 - 2 * beta / K) * model.B[0] + \
        n * model.C[0] + n * (n - 1) * model.D[0]

model = pyo.ConcreteModel()

model.A = pyo.Var(np.arange(1), domain=pyo.NonNegativeReals, initialize=0.5)
model.B = pyo.Var(np.arange(1), domain=pyo.NonNegativeReals, initialize=0.5)
model.C = pyo.Var(np.arange(1), domain=pyo.NonNegativeReals, initialize=0.5)
model.D = pyo.Var(np.arange(1), domain=pyo.NonNegativeReals, initialize=0.5)

model.obj = pyo.Objective(rule=upper_bound, sense=pyo.maximize)
model.costr = pyo.ConstraintList()

model.costr.add(model.A[0] >= model.B[0])
model.costr.add(model.C[0] >= model.D[0])
model.costr.add(model.A[0] >= model.C[0])
model.costr.add(model.B[0] >= model.D[0])
model.costr.add(model.A[0] <= 1)
model.costr.add(model.B[0] <= 1)
model.costr.add(model.C[0] <= 1)
model.costr.add(model.D[0] <= 1)

In [ ]:
solver = pyo.SolverFactory("ipopt")
out = solver.solve(model)
print(out)

In [ ]:
model.C.extract_values()

# Worst-Case for $M_{r+1, n}$

In [ ]:
### Optimize the Plug-in Upper Bound

In [ ]:
from scipy.special import binom

J = 20
pis = np.random.uniform(size=J)
pis /= np.sum(pis)


r = 1
n = 5000

a = binom(n, r+1) * (r / (n-1)) ** r * (1 - r / (n-1)) ** (n - r - 1)
c = (r / (n-2)) ** (2 * r) * (1 - 2 * r / (n - 2)) ** (n - 2*r - 2)

In [ ]:
def sp(a, b):
    out = 0
    for j in range(len(b)):
        out += a[j] * b[j]
    return out


def ub(m):
    pib = sp(m.beta, pis)

    return (n / (r + 1))**2 * pib**2 - 2 * n / (r+1) * a * pib + a + c

In [ ]:
model = pyo.ConcreteModel()
model.beta = pyo.Var(np.arange(J), domain=pyo.NonNegativeReals, initialize=0.2)

model.obj = pyo.Objective(
    rule=lambda m: ub(m), sense=pyo.minimize)

model.costr = pyo.ConstraintList()

for i in range(J):
    model.costr.add(model.beta[i] <= 1)
solver = pyo.SolverFactory("ipopt")
out = solver.solve(model)
print(out)

In [ ]:
plt.scatter(pis, model.beta.extract_values().values())

In [ ]:
model.beta.extract_values()

# Minimax Estimation

In [ ]:
def inner_opt(m):
    return m.z[0]


stupid_init = np.zeros(J)
stupid_init[:2] = 0.5

model = pyo.ConcreteModel()
model.beta = pyo.Var(np.arange(J), domain=pyo.NonNegativeReals, initialize=stupid_init)
model.pis = pyo.Var(np.arange(J), domain=pyo.NonNegativeReals, initialize=stupid_init)
model.z = pyo.Var(np.arange(1), domain=pyo.Reals, initialize=np.ones(1) * 1e6)

model.obj = pyo.Objective(rule=inner_opt)
model.costr = pyo.ConstraintList()

model.costr.add(pyo.summation(model.pis) == 1)

# minimax trick
model.costr.add(model.z[0] >= ub(model))
    
solver = pyo.SolverFactory("multistart")
out = solver.solve(model, suppress_unbounded_warning=True)
print(out)

In [ ]:
model.pis.extract_values()

In [ ]:
model.beta.extract_values()

In [6]:
r = 11
n = 3000

K = n / (r-1)
print(K)

300.0


In [ ]:
def f1(m):
    return (r / (m-2)) ** (2 * r) * (1 - 2 * r / (m - 2)) ** (m - 2*r - 2)


def f2(m):
    return m / r * (m / r - 1) * (r / m) ** (2 * r + 2) * (1 - 2 * r / (m)) ** (m - 2*r - 2)

In [ ]:
n_grid = np.linspace(1000, 10000000, 1000)
plt.plot(n_grid, np.log(np.abs(f1(n_grid) - f2(n_grid))))